In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from module.genome import Genome
import datetime
import random
import matplotlib.pyplot as plt

#%%
N_POPULATION = 250                      # 세대당 생성수
N_BEST = 15                             # 베스트 수
N_CHILDREN = 10                         # 자손 유전자 수
PROB_MUTATION = 0.4                    # 돌연변이
REVERSE = True                        # 배열 순서 (False: ascending order, True: descending order)

score_ini = 10                         # 초기 점수
input_length = 125                     # 입력 데이터 길이
output_length_1 = 5 * 2                # Event (CHECK_1~4, PROCESS)
output_length_2 = 12 * 2                   # MOL(0~5.5, step:0.5)
h1 = 50                                # 히든레이어1 노드 수
h2 = 50                                # 히든레이어2 노드 수
h3 = 50                                # 히든레이어3 노드 수
EPOCHS = 300                            # 반복 횟수

mut_mean = 0
mut_stddev = 0.2

genomes = []
best_genome_weights = []

for _ in range(N_POPULATION):
    genome = Genome(score_ini, input_length, output_length_1, output_length_2, h1, h2, h3)
    genomes.append(genome)



In [2]:
#%%
n_gen = 1
high_score_history = []
mean_score_history = []

best_gen = None
best_score_ever = 0
#%%
while n_gen <= EPOCHS:
    print('EPOCH', n_gen, datetime.datetime.now())
    for idx, genome in enumerate(genomes):
        genome.predict()

    genomes.sort(key=lambda x: x.score, reverse=REVERSE)

    # 평균
    mean_score = 0
    for i in range(N_POPULATION):
        mean_score += genomes[i].score
    mean_score /= N_POPULATION
    mean_score_history.append([n_gen, mean_score])

    # 최고
    high_score_history.append([n_gen, genomes[0].score])

    # 최고 모델 저장
    if genomes[0].score > best_score_ever:
        best_score_ever = genomes[0].score
        best_gen = Genome(score_ini, input_length, output_length_1, output_length_2, h1, h2, h3)
        best_gen.copy_weights(genomes[0])

    print('EPOCH #%s\tHistory Best Score: %s\tBest Score: %s\tMean Score: %s' % (n_gen, best_score_ever, genomes[0].score, mean_score))

    # best model weight 저장
    best_genome_weights = []
    for i in range(N_BEST):
        best_genome_weights.append(genomes[0].get_weights())

    # CHILDREN 생성
    for i in range(N_CHILDREN):
        new_weights = deepcopy(best_genome_weights[0])
        a_weights = random.choice(best_genome_weights)
        b_weights = random.choice(best_genome_weights)

        # model_num, weight_num
        for m in range(len(new_weights)):
            for w in range(len(new_weights[m])):
                for j in range(new_weights[m][w].shape[0]):
                    if len(new_weights[m][w].shape) > 1:
                        cut = np.random.randint(new_weights[m][w].shape[1])
                        new_weights[m][w][j, :cut] = a_weights[m][w][j, :cut]
                        new_weights[m][w][j, cut:] = b_weights[m][w][j, cut:]
                    else:
                        new_weights[m][w][j] = random.choice((a_weights[m][w][j], b_weights[m][w][j]))

        best_genome_weights.append(new_weights)

    # 모델 초기화 and 돌연변이

    for i in range(len(best_genome_weights)):
        genomes[i].set_weights(best_genome_weights[i])

    for i in range(len(best_genome_weights), N_POPULATION):
        bgw = best_genome_weights[i % len(best_genome_weights)]
        new_weights = deepcopy(bgw)

        for m in range(len(new_weights)):
            for w in range(len(new_weights[m])):
                if np.random.uniform(0, 1) < PROB_MUTATION:
                    new_weights[m][w] += new_weights[m][w]\
                                         * np.random.normal(mut_mean, mut_stddev, size=new_weights[m][w].shape)\
                                         * np.random.randint(0, 2, new_weights[m][w].shape)

        genomes[i].set_weights(new_weights)

    n_gen += 1


EPOCH 1 2020-07-03 17:29:21.778448
EPOCH #1	History Best Score: 83.72030513507644	Best Score: 83.72030513507644	Mean Score: 66.08265322020978
EPOCH 2 2020-07-03 17:30:19.944732
EPOCH #2	History Best Score: 85.3860362393217	Best Score: 85.3860362393217	Mean Score: 80.14505072466662
EPOCH 3 2020-07-03 17:31:08.328642
EPOCH #3	History Best Score: 85.76291910221644	Best Score: 85.76291910221644	Mean Score: 80.9818819440242
EPOCH 4 2020-07-03 17:31:56.456183
EPOCH #4	History Best Score: 86.2300850700274	Best Score: 86.2300850700274	Mean Score: 80.48706290930217
EPOCH 5 2020-07-03 17:32:42.675092
EPOCH #5	History Best Score: 86.2300850700274	Best Score: 86.2300850700274	Mean Score: 80.85267736398735
EPOCH 6 2020-07-03 17:33:29.728222
EPOCH #6	History Best Score: 86.2300850700274	Best Score: 86.2300850700274	Mean Score: 80.70561505804918
EPOCH 7 2020-07-03 17:34:14.538108
EPOCH #7	History Best Score: 86.2300850700274	Best Score: 86.2300850700274	Mean Score: 80.87107446784
EPOCH 8 2020-07-03 1

KeyboardInterrupt: 

In [ ]:
#%%
n_gen = 1
high_score_history = []
mean_score_history = []

best_gen = None
best_score_ever = 0
#%%
while n_gen <= EPOCHS:
    print('EPOCH', n_gen, datetime.datetime.now())
    for idx, genome in enumerate(genomes):
        genome.predict()

    genomes.sort(key=lambda x: x.score, reverse=REVERSE)

    # 평균
    mean_score = 0
    for i in range(N_POPULATION):
        mean_score += genomes[i].score
    mean_score /= N_POPULATION
    mean_score_history.append([n_gen, mean_score])

    # 최고
    high_score_history.append([n_gen, genomes[0].score])

    # 최고 모델 저장
    if genomes[0].score > best_score_ever:
        best_score_ever = genomes[0].score
        best_gen = Genome(score_ini, input_length, output_length_1, output_length_2, h1, h2, h3)
        best_gen.copy_weights(genomes[0])

    print('EPOCH #%s\tHistory Best Score: %s\tBest Score: %s\tMean Score: %s' % (n_gen, best_score_ever, genomes[0].score, mean_score))

    # best model weight 저장
    best_genome_weights = []
    for i in range(N_BEST):
        best_genome_weights.append(genomes[0].get_weights())

    # CHILDREN 생성
    for i in range(N_CHILDREN):
        new_weights = deepcopy(best_genome_weights[0])
        a_weights = random.choice(best_genome_weights)
        b_weights = random.choice(best_genome_weights)

        # model_num, weight_num
        for m in range(len(new_weights)):
            for w in range(len(new_weights[m])):
                for j in range(new_weights[m][w].shape[0]):
                    if len(new_weights[m][w].shape) > 1:
                        cut = np.random.randint(new_weights[m][w].shape[1])
                        new_weights[m][w][j, :cut] = a_weights[m][w][j, :cut]
                        new_weights[m][w][j, cut:] = b_weights[m][w][j, cut:]
                    else:
                        new_weights[m][w][j] = random.choice((a_weights[m][w][j], b_weights[m][w][j]))

        best_genome_weights.append(new_weights)

    # 모델 초기화 and 돌연변이

    for i in range(len(best_genome_weights)):
        genomes[i].set_weights(best_genome_weights[i])

    for i in range(len(best_genome_weights), N_POPULATION):
        bgw = best_genome_weights[i % len(best_genome_weights)]
        new_weights = deepcopy(bgw)

        for m in range(len(new_weights)):
            for w in range(len(new_weights[m])):
                if np.random.uniform(0, 1) < PROB_MUTATION:
                    new_weights[m][w] += new_weights[m][w]\
                                         * np.random.normal(mut_mean, mut_stddev, size=new_weights[m][w].shape)\
                                         * np.random.randint(0, 2, new_weights[m][w].shape)

        genomes[i].set_weights(new_weights)

    n_gen += 1


EPOCH 1 2020-07-03 20:14:52.419567
EPOCH #1	History Best Score: 87.70624650738553	Best Score: 87.70624650738553	Mean Score: 82.43092379887091
EPOCH 2 2020-07-03 20:15:34.758326
EPOCH #2	History Best Score: 87.76642623244938	Best Score: 87.76642623244938	Mean Score: 82.06719488013057
EPOCH 3 2020-07-03 20:16:16.716803
EPOCH #3	History Best Score: 87.76642623244938	Best Score: 87.76642623244938	Mean Score: 81.96751351040817
EPOCH 4 2020-07-03 20:16:58.864111
EPOCH #4	History Best Score: 87.76642623244938	Best Score: 87.76642623244938	Mean Score: 82.12449277637471
EPOCH 5 2020-07-03 20:17:42.866810
EPOCH #5	History Best Score: 87.78630283774638	Best Score: 87.78630283774638	Mean Score: 81.96944973876246
EPOCH 6 2020-07-03 20:18:26.615388
EPOCH #6	History Best Score: 87.78630283774638	Best Score: 87.78630283774638	Mean Score: 82.25042049987664
EPOCH 7 2020-07-03 20:19:09.404891
EPOCH #7	History Best Score: 87.89514614527494	Best Score: 87.89514614527494	Mean Score: 81.79552841141073
EPOCH 

EPOCH 59 2020-07-03 20:56:12.225899
EPOCH #59	History Best Score: 89.319619192506	Best Score: 89.319619192506	Mean Score: 82.46981811155042
EPOCH 60 2020-07-03 20:56:55.393015
EPOCH #60	History Best Score: 89.319619192506	Best Score: 89.319619192506	Mean Score: 83.00619161723374
EPOCH 61 2020-07-03 20:57:39.220089
EPOCH #61	History Best Score: 89.319619192506	Best Score: 89.319619192506	Mean Score: 82.69212155266082
EPOCH 62 2020-07-03 20:58:22.345111
EPOCH #62	History Best Score: 89.319619192506	Best Score: 89.319619192506	Mean Score: 82.4060430901455
EPOCH 63 2020-07-03 20:59:05.107149
EPOCH #63	History Best Score: 89.319619192506	Best Score: 89.319619192506	Mean Score: 82.75789660663037
EPOCH 64 2020-07-03 20:59:48.689128
EPOCH #64	History Best Score: 89.319619192506	Best Score: 89.319619192506	Mean Score: 82.3945979650903
EPOCH 65 2020-07-03 21:00:33.529294
EPOCH #65	History Best Score: 89.319619192506	Best Score: 89.319619192506	Mean Score: 82.85446383026179
EPOCH 66 2020-07-03 21

EPOCH 118 2020-07-03 21:38:07.954580
EPOCH #118	History Best Score: 89.319619192506	Best Score: 89.319619192506	Mean Score: 82.26017422842986
EPOCH 119 2020-07-03 21:38:50.368706
EPOCH #119	History Best Score: 89.319619192506	Best Score: 89.319619192506	Mean Score: 82.8267720374565
EPOCH 120 2020-07-03 21:39:32.421105
EPOCH #120	History Best Score: 89.38384551177344	Best Score: 89.38384551177344	Mean Score: 82.41977093671653
EPOCH 121 2020-07-03 21:40:15.578959
EPOCH #121	History Best Score: 89.38384551177344	Best Score: 89.38384551177344	Mean Score: 82.69085771760655
EPOCH 122 2020-07-03 21:40:57.827969
EPOCH #122	History Best Score: 89.38384551177344	Best Score: 89.38384551177344	Mean Score: 82.99860516736443
EPOCH 123 2020-07-03 21:41:40.370794
EPOCH #123	History Best Score: 89.38384551177344	Best Score: 89.38384551177344	Mean Score: 83.07059980344587
EPOCH 124 2020-07-03 21:42:22.684379
EPOCH #124	History Best Score: 89.38384551177344	Best Score: 89.38384551177344	Mean Score: 83.31

EPOCH #174	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 82.40824083318311
EPOCH 175 2020-07-03 22:18:05.680633
EPOCH #175	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 81.8278838690135
EPOCH 176 2020-07-03 22:18:48.384899
EPOCH #176	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 81.88614034849496
EPOCH 177 2020-07-03 22:19:30.652508
EPOCH #177	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 82.28522339614683
EPOCH 178 2020-07-03 22:20:13.477001
EPOCH #178	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 81.45102433701383
EPOCH 179 2020-07-03 22:20:55.422792
EPOCH #179	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 82.46520082590831
EPOCH 180 2020-07-03 22:21:38.210928
EPOCH #180	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 82.17749011254092
EPOCH 181 2020-0

EPOCH 231 2020-07-03 22:57:47.942853
EPOCH #231	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 81.93486130461186
EPOCH 232 2020-07-03 22:58:29.934724
EPOCH #232	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 81.7471640311476
EPOCH 233 2020-07-03 22:59:12.493096
EPOCH #233	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 81.89664434608895
EPOCH 234 2020-07-03 22:59:53.157181
EPOCH #234	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 82.71983961576406
EPOCH 235 2020-07-03 23:00:36.581366
EPOCH #235	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 82.10826701746167
EPOCH 236 2020-07-03 23:01:18.476834
EPOCH #236	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Score: 81.47571913588685
EPOCH 237 2020-07-03 23:02:01.655914
EPOCH #237	History Best Score: 89.38795078343625	Best Score: 89.38795078343625	Mean Scor

In [ ]:
#%%
# Score Graph
high_score_history = np.array(high_score_history)
mean_score_history = np.array(mean_score_history)
plt.plot(high_score_history[:,0], high_score_history[:,1], '-o', label='High')
plt.plot(mean_score_history[:,0], mean_score_history[:,1], '-o', label='Mean')
plt.legend()
plt.xlim(0, EPOCHS)
plt.ylim(bottom=0)
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.show()


In [ ]:

#%%
best_gen.make_submission_file("submit.csv")